##### 検索内容の設定

In [1]:
#レース名
race_name = "高松宮記念"
dir = 'takamatsu/racepage'

In [2]:
#期間
year = 2006
month = 1

end_year = 2022
end_month = 12

過去のデータをデータベースからスクレイピングするコード

In [3]:
#インポート文
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import time
import requests

In [4]:
#ドライバーの設定
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

C:\Users\kawam\AppData\Local\Temp\ipykernel_16668\2084027807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


##### ドライバーの起動

In [5]:
#URLの設定とドライバー起動
URL = "https://db.netkeiba.com/?pid=race_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

##### 検索画面で要素の選択

In [6]:
#レース名を選択
race_name_element =driver.find_element(By.CSS_SELECTOR,"#db_search_detail_form > form > table > tbody > tr:nth-child(1) > td > input")
race_name_element.send_keys(race_name)

In [7]:
#期間を選択
year_element =driver.find_element(By.NAME,"start_year")
year_select = Select(year_element)
year_select.select_by_value(str(year))
month_element = driver.find_element(By.NAME,"start_mon")
month_select = Select(month_element)
month_select.select_by_value(str(month))
end_year_element = driver.find_element(By.NAME,"end_year")
end_year_select = Select(end_year_element)
end_year_select.select_by_value(str(end_year))
end_mon_element = driver.find_element(By.NAME,"end_mon")
end_mon_select = Select(end_mon_element)
end_mon_select.select_by_value(str(end_month))

In [8]:
#中央競馬場をチェック
for i in range(1,11):
    terms = driver.find_element(By.ID,"check_Jyo_"+ str(i).zfill(2))
    terms.click()
#1札幌　2函館　3福島　4新潟　5東京　6中山　7中京　8京都　9阪神　10小倉　

In [9]:
#グレードをチェック
for i in range(1,2):
    terms = driver.find_element(By.ID,"check_grade_"+ str(i))
    terms.click()
#1:G1　2:G2　3:G3　4:OP　5:3勝(1600万)　6:2勝(1000万)　7:1勝(500万)　8:新馬　9:未勝利　10:未出走　

In [10]:
# 表示件数を選択(20,50,100の中から最大の100へ)
list_element = driver.find_element(By.NAME,'list')
list_select = Select(list_element)
list_select.select_by_value("100")

In [11]:
# フォームを送信
frm = driver.find_element(By.CSS_SELECTOR,"#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

##### レース一覧画面

In [12]:
#レースデータをテキストに保存
TEXT_DIR = "text/"+dir
if not os.path.isdir(TEXT_DIR):
    os.makedirs(TEXT_DIR)

#with open(TEXT_DIR+"/"+str(year)+"-"+str(month)+".txt", mode='w') as f:
with open(TEXT_DIR+"/"+".txt", mode='w') as f:
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        all_rows = driver.find_element(By.CLASS_NAME,'race_table_01').find_elements(By.TAG_NAME,"tr")
        for row in range(1, len(all_rows)):
            race_href=all_rows[row].find_elements(By.TAG_NAME,"td")[4].find_element(By.TAG_NAME,"a").get_attribute("href")
            f.write(race_href+"\n")
        try:
            target = driver.find_elements(By.LINK_TEXT,"次")[0]
            driver.execute_script("arguments[0].click();", target) 
        except IndexError:
            break

KeyboardInterrupt: 

In [ ]:
#テキストデータからHTMLデータに保存
#HTML_DIR = "html/"+dir+"/"+str(year)+"/"+str(month)
HTML_DIR = "html/"+dir+"/"
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
        
#with open(TEXT_DIR+"/"+str(year)+"-"+str(month)+".txt", "r") as f:
with open(TEXT_DIR+"/"+".txt", "r") as f:
    urls = f.read().splitlines()
    for url in urls:
        list = url.split("/")
        race_id = list[-2]
        save_file_path = HTML_DIR+"/"+race_id+'.html'
        response = requests.get(url)
        response.encoding = response.apparent_encoding
        html = response.text
        time.sleep(5)
        with open(save_file_path, 'w') as file:
            file.write(html)

In [ ]:
#HTMLデータからスクレイピングする関数を定義
def get_rade_and_horse_data_by_html(race_id, html):
    race_list = [race_id]
    horse_list_list = []
    soup = BeautifulSoup(html, 'html.parser')

    # race基本情報
    data_intro = soup.find("div", class_="data_intro")
    race_list.append(data_intro.find("dt").get_text().strip("\n")) # race_round
    race_list.append(data_intro.find("h1").get_text().strip("\n")) # race_title
    race_details1 = data_intro.find("p").get_text().strip("\n").split("\xa0/\xa0")
    race_list.append(race_details1[0]) # race_course
    race_list.append(race_details1[1]) # weather
    race_list.append(race_details1[2]) # ground_status
    race_list.append(race_details1[3]) # time
    race_details2 = data_intro.find("p", class_="smalltxt").get_text().strip("\n").split(" ")
    race_list.append(race_details2[0]) # date
    race_list.append(race_details2[1]) # where_racecourse


    result_rows = soup.find("table", class_="race_table_01 nk_tb_common").findAll('tr') # レース結果
    # 上位3着の情報
    race_list.append(len(result_rows)-1) # total_horse_number
    for i in range(1,4):
        row = result_rows[i].findAll('td')
        race_list.append(row[1].get_text()) # frame_number_first or second or third
        race_list.append(row[2].get_text()) # horse_number_first or second or third


    # 払い戻し(単勝・複勝・三連複・3連単)
    pay_back_tables = soup.findAll("table", class_="pay_table_01")

    pay_back1 = pay_back_tables[0].findAll('tr') # 払い戻し1(単勝・複勝)
    race_list.append(pay_back1[0].find("td", class_="txt_r").get_text()) #tansyo
    hukuren = pay_back1[1].find("td", class_="txt_r")
    tmp = []
    for string in hukuren.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # 枠連
    try:
        race_list.append(pay_back1[2].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")

    # 馬連
    try:
        race_list.append(pay_back1[3].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")



    pay_back2 = pay_back_tables[1].findAll('tr') # 払い戻し2(三連複・3連単)

    # wide 1&2
    wide = pay_back2[0].find("td", class_="txt_r")
    tmp = []
    for string in wide.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # umatan
    race_list.append(pay_back2[1].find("td", class_="txt_r").get_text()) #umatan

    race_list.append(pay_back2[2].find("td", class_="txt_r").get_text()) #renhuku3
    try:
        race_list.append(pay_back2[3].find("td", class_="txt_r").get_text()) #rentan3
    except IndexError:
        race_list.append("0")

    # horse data
    for rank in range(1, len(result_rows)):
        horse_list = [race_id]
        result_row = result_rows[rank].findAll("td")
        # rank
        horse_list.append(result_row[0].get_text())
        # frame_number
        horse_list.append(result_row[1].get_text())
        # horse_number
        horse_list.append(result_row[2].get_text())
        # horse_id
        horse_list.append(result_row[3].find('a').get('href').split("/")[-2])
        # sex_and_age
        horse_list.append(result_row[4].get_text())
        # burden_weight
        horse_list.append(result_row[5].get_text())
        # rider_id
        horse_list.append(result_row[6].find('a').get('href').split("/")[-2])
        # goal_time
        horse_list.append(result_row[7].get_text())
        # goal_time_dif
        horse_list.append(result_row[8].get_text())
        # time_value(premium)
        horse_list.append(result_row[9].get_text())
        # half_way_rank
        horse_list.append(result_row[10].get_text())
        # last_time(上り)
        horse_list.append(result_row[11].get_text())
        # odds
        horse_list.append(result_row[12].get_text())
        # popular
        horse_list.append(result_row[13].get_text())
        # horse_weight
        horse_list.append(result_row[14].get_text())
        # tame_time(premium)
        horse_list.append(result_row[15].get_text())
        # 16:コメント、17:備考
        # tamer_id
        horse_list.append(result_row[18].find('a').get('href').split("/")[-2])
        # owner_id
        horse_list.append(result_row[19].find('a').get('href').split("/")[-2])

        horse_list_list.append(horse_list)

    return race_list, horse_list_list

In [ ]:
#スクレイピングするカラムの設定
race_data_columns=[
    'race_id',
    'race_round',
    'race_title',
    'race_course',
    'weather',
    'ground_status',
    'time',
    'date',
    'where_racecourse',
    'total_horse_number',
    'frame_number_first',
    'horse_number_first',
    'frame_number_second',
    'horse_number_second',
    'frame_number_third',
    'horse_number_third',
    'tansyo',
    'hukusyo_first',
    'hukusyo_second',
    'hukusyo_third',
    'wakuren',
    'umaren',
    'wide_1_2',
    'wide_1_3',
    'wide_2_3',
    'umatan',
    'renhuku3',
    'rentan3'
    ]

horse_data_columns=[
    'race_id',
    'rank',
    'frame_number',
    'horse_number',
    'horse_id',
    'sex_and_age',
    'burden_weight',
    'rider_id',
    'goal_time',
    'goal_time_dif',
    'time_value',
    'half_way_rank',
    'last_time',
    'odds',
    'popular',
    'horse_weight',
    'tame_time',
    'tamer_id',
    'owner_id'
]

In [ ]:
#csvの保存場所を設定
CSV_DIR = "csv/"+dir
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
# save_race_csv = CSV_DIR+"/race-"+str(year)+"-"+str(month)+".csv"
# horse_race_csv = CSV_DIR+"/horse-"+str(year)+"-"+str(month)+".csv"
save_race_csv = CSV_DIR+"/race"+".csv"
horse_race_csv = CSV_DIR+"/horse"+".csv"

In [ ]:
#上記で定義した関数を使って、各要素をデータフレームに保存
race_df = pd.DataFrame(columns=race_data_columns)
horse_df = pd.DataFrame(columns=horse_data_columns)
#html_dir = "html"+"/"+str(year)+"/"+str(month)
if os.path.isdir(HTML_DIR):
    file_list = os.listdir(HTML_DIR)
    for file_name in file_list:
        with open(HTML_DIR+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            race_id = list[-2]
            race_list, horse_list_list = get_rade_and_horse_data_by_html(race_id, html) 
            for horse_list in horse_list_list:
                horse_se = pd.Series( horse_list, index=horse_df.columns)
                horse_df = pd.concat([horse_df, horse_se.to_frame().T], ignore_index=True)
            race_se = pd.Series(race_list, index=race_df.columns )
            race_df = pd.concat([race_df, race_se.to_frame().T], ignore_index=True)

In [ ]:
#dfをcsvに書き出し
race_df.to_csv(save_race_csv, header=True, index=False)
horse_df.to_csv(horse_race_csv, header=True, index=False)